# 04 - Variography (3D + dominios + direcciones)

Este notebook usa las funciones ampliadas de `src/variography.py` para:

- Variografía 3D con direcciones (azimut/dip) y bandas.
- Separación por dominio.
- Ajuste con checks básicos.
- Exportación con trazabilidad.


In [ ]:
from __future__ import annotations

from pathlib import Path

import pandas as pd

from src.variography import (
    _hash_dataframe,
    compute_variograms_by_domain,
    export_variogram_results,
    fit_variogram_results,
)

ROOT = Path.cwd()
if ROOT.name == 'notebooks':
    ROOT = ROOT.parent

DATA = ROOT / 'data_sample' / 'sample_drillholes.csv'
OUT = ROOT / 'outputs' / 'tables'


In [ ]:
df = pd.read_csv(DATA)
df.head()


In [ ]:
coords = ('X', 'Y', 'Z')
vcol = 'grade'
domain_col = 'domain'

base_params = {
    'lag': 50.0,
    'nlag': 12,
    'atol': 22.5,
    'bandwh': 100.0,
}

directions = [
    {'azm': 0.0, 'dip': 0.0},
    {'azm': 90.0, 'dip': 0.0},
    {'azm': 0.0, 'dip': 90.0},
]

results = compute_variograms_by_domain(
    df,
    coords=coords,
    vcol=vcol,
    params=base_params,
    directions=directions,
    domain_col=domain_col,
)

variance = float(df[vcol].var(ddof=1))
results = fit_variogram_results(results, data_variance=variance, min_pairs=30)


In [ ]:
data_hash = _hash_dataframe(df)
exported = export_variogram_results(results, OUT, data_hash=data_hash)
exported
